## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generating Synthetic Data (Customer Reviews) using GenAI


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook will use GenAI to generate fake customer reviews and then score the review's sentiment.

- Quick Notes:
    - Why synthetic data:
        - No licensing fees
        - LLMs can read your schema and description fields which means it understands what data to generate.
        - LLMs can generate fake names, addresses, latitude/longitude based upon a fake address.
        - LLMs can positive, neutrel and negative reviews.
    - In order to generate synthetic data we need to make sure we can handle the following items:
        - Have a unique primary key (INTs cannot be deplicated, UUIDs are easier to for synthetic generation)
        - Ensure that foreign key data is valid

- Notebook Logic:
    1. Get a list of valid customer foreign keys
    2. Get a list of valid location foreign keys
    3. Get the table schema from BigQuery (this also contains the description for each field)
    4. Loop for the number of review to generate
        - Get the current maximum primary key and add one
        - Create our LLM prompt:
            - Determine if we are writing a negative or positive/neutral review 
            - Provide a list of themes we want reviews based upon
            - Provide the prompt with a suggust range of reivew text words
            - Provide the prompt with our foreign keys
            - Provide the prompt with our primary key.  And since we are asking for more the one review to be generated at a time, the LLM is smart enough to increment it for each row.
            - Provide the prompt that we want a single INSERT..INTO statemend versus multiple.
    5. Execute the generated customer review SQL statement against BigQuery.  This will insert the data.
    6. For the reviews generated
        - Create a prompt asking to determine the sentiment (Positive, Neutral or Negative)
        - Update BigQuery with the sentiment

## Initialize Python

In [ ]:
import pandas as pd
import json
import bigframes.pandas as bf
#from bigframesllm import BigFramesLLM
from bigframes.ml.llm import GeminiTextGenerator

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
PROJECT_ID = "${project_id}"
REGION = "us"
DATASET_ID = "${bigquery_data_beans_curated_dataset}"
CONNECTION_NAME = "vertex-ai"

connection = f"{PROJECT_ID}.{REGION}.{CONNECTION_NAME}"

In [ ]:
# bf.reset_session() # if you need to change the region
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

In [ ]:
session = bf.get_global_session()

llm_model = GeminiTextGenerator(session=session, connection_name=connection)

## Supporting Functions

In [ ]:
def PrettyPrintJson(json_string):
  json_object = json.loads(json_string)
  json_formatted_str = json.dumps(json_object, indent=2)
  print(json_formatted_str)
  return json.dumps(json_object)

In [ ]:
def LLM(prompt, isOutputJson, max_output_tokens=1024, temperature=0, top_p=0, top_k=1):
  print()
  print("Prompt: ", prompt)
  print()
  df_prompt = pd.DataFrame(
          {
              "prompt": [prompt],
          })
  bf_df_prompt = bf.read_pandas(df_prompt)
  prediction = llm_model.predict(bf_df_prompt,
                                 max_output_tokens=max_output_tokens,
                                 temperature=temperature, # 0 to 1 (1 random)
                                 top_p=top_p, # 0 to 1 (1 random)
                                 top_k=top_k, # (1 to 40 random)
                                 ).to_pandas()
  try:
    # Remove common LLM output mistakes
    result = prediction['ml_generate_text_llm_result'][0]

    result = result.replace("```json\n","")
    result = result.replace("```JSON\n","")
    result = result.replace("```json","")
    result = result.replace("```JSON","")
    result = result.replace("```sql\n","")
    result = result.replace("```SQL\n","")
    result = result.replace("```sql","")
    result = result.replace("```sql:  bigquery","")     
    result = result.replace("```SQL","")
    result = result.replace("```","")

    if isOutputJson:
      json_string = PrettyPrintJson(result)
      json_string = json_string.replace("'","\\'")
      json_string = json_string.strip()
      return json_string
    else:
      # result = result.replace("'","\\'")
      result = result.strip()
      return result

  except:
    print("Error (raw): ", prediction['ml_generate_text_llm_result'][0])
    print("Error (result): ", result)


In [ ]:
def GetTableSchema(dataset_name, table_name):
  import io

  dataset_ref = client.dataset(dataset_name, project=PROJECT_ID)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetForeignKeys(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(CAST({field_name} AS STRING), "," ORDER BY {field_name}) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  WHERE {field_name} <= 10 -- demo hack since the database is fully populated
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetDistinctValues(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetStartingValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetMaximumValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{PROJECT_ID}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def RunQuery(sql):
  import time

  #return True # return early for now

  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

## Synthetic Data Generation

### Create customer reviews

In [ ]:
rows_of_data_to_generate = 3

table_name = "customer"
field_name = "customer_id"
customer_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "location"
field_name = "location_id"
location_ids = GetForeignKeys(DATASET_ID, table_name, field_name)

table_name = "customer_review"
primary_key = "customer_review_id"

schema = GetTableSchema(DATASET_ID, table_name)

In [ ]:
# location_ids='1,11,21'

In [ ]:
import random
loop_count = 1 # can be set higher
loop_index = 1

while loop_index <= loop_count:
  print(f"loop_index: {loop_index} | loop_count: {loop_count}")
  starting_value = GetStartingValue(DATASET_ID, table_name, primary_key)

  if random.random() < .25:
    prompt=f"""
    You are a database engineer and need write a single BigQuery SQL statement.
    You to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your brewed coffee.
    Write a negative in first person based upon the following: "Bad Service","Long Wait Time","Slow Service","Dirty","Overpriced","Overcrowded","Noisy Location","Lack of Allergan Information","Inconsistent Quality","Lack of Seating","No Flavor","Too weak","Too strong","Too bitter","Limited Menu"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - Valid values for location_id are: {location_ids}
    - The review_datetime is a date and should be within the past 5 years.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Only generate data for these fields: customer_review_id, customer_id, location_id, review_datetime, review_text, social_media_source, social_media_handle
    - Timestamps should use this format: 2020-06-02 23:57:12.120174 UTC.

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """
  else:
    prompt=f"""
    You are a database engineer and need write a single BigQuery SQL statement.
    You to generate data for a table for the below schema.
    You need to generate reviews for customers who have purchased your brewed coffee.
    Write a positive or neutral review in first person based upon the following: "Good Service","Short Wait Time","Fast Service","Clean","Good value","Cozy Seating Areas","Quite Location","Variety of Milk Alternatives","Consistent Quality","Lots of places to sit","Lots of Flavor","Good Taste","Good Selection"
    - The schema is for a Google Cloud BigQuery Table.
    - The table name is "{PROJECT_ID}.{DATASET_ID}.{table_name}".
    - Read the description of each field for valid values.
    - Do not preface the response with any special characters or 'sql'.
    - Generate {rows_of_data_to_generate} insert statements for this table.
    - Valid values for customer_id are: {customer_ids}
    - The review_datetime is a date and should be within the past 5 years.
    - The response for each question should be 20 to 100 words.
    - The starting value of the field {primary_key} is {starting_value}.
    - Only generate data for these fields: customer_review_id, customer_id, location_id, review_datetime, review_text, social_media_source, social_media_handle
    - Timestamps should use this format: 2020-06-02 23:57:12.120174 UTC.

    Examples:
    Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
    Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

    Schema: {schema}
    """

  llm_valid_execution = False
  temperature=.8
  while llm_valid_execution == False:
    try:
      sql = LLM(prompt, False, max_output_tokens=1024, temperature=temperature, top_p=1, top_k=40)
      print("---------------------------------")
      print("sql: ", sql)
      print("---------------------------------")
      llm_valid_execution = RunQuery(sql)
      loop_index = loop_index + 1
    except Exception as error:
      temperature = temperature - .1
      print("An error occurred:", error)

### Score the Sentiment

In [ ]:
sql = """SELECT customer_review_id,
                review_text
          FROM `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
         WHERE review_sentiment IS NULL
        ORDER BY customer_review_id
"""

# Fields to update
# review_sentiment,
# gen_ai_recommended_action,
# gen_ai_reponse,
# human_response,
# response_sent_action,
# response_sent_date

df_process = client.query(sql).to_dataframe()

for row in df_process.itertuples():
  customer_review_id = row.customer_review_id
  review_text = row.review_text

  llm_valid_execution = False
  while llm_valid_execution == False:
    try:
      prompt=f"""
      For the given review classify the sentiment as Positive, Neutral or Negative.
      Only return one of these words: "Positive", "Neutral", "Negative"
      Review: {review_text}
      """
      review_sentiment = LLM(prompt, False, max_output_tokens=10, temperature=0, top_p=0, top_k=1)

      sql = f"""UPDATE `${project_id}.${bigquery_data_beans_curated_dataset}.customer_review`
                  SET review_sentiment = '{review_sentiment}'
                WHERE customer_review_id = {customer_review_id}
      """

      print (sql)

      llm_valid_execution = RunQuery(sql)
      llm_valid_execution = True
    except Exception as error:
      print("An error occurred:", error)